In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /home/admin.alahmid@lending.local/.cache/huggingface/token
Login successful


In [3]:
from datasets import load_dataset

ds = load_dataset("scene_parse_150", split="train[:70]")

No config specified, defaulting to: scene_parse_150/scene_parsing
Found cached dataset scene_parse_150 (/home/admin.alahmid@lending.local/.cache/huggingface/datasets/scene_parse_150/scene_parsing/1.0.0/d998c54e1b5c5bad12b4d2ec7e1a5f74eee4c153bc1b089a0001677ae9b3fd75)


In [4]:
ds = ds.train_test_split(test_size=0.3)
train_ds = ds["train"]
test_ds = ds["test"]

In [5]:
train_ds[0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=765x512>,
 'annotation': <PIL.PngImagePlugin.PngImageFile image mode=L size=765x512>,
 'scene_category': 46}

In [6]:
import json
from huggingface_hub import cached_download, hf_hub_url
repo_id = "huggingface/label-files"
filename = "ade20k-id2label.json"
id2label = json.load(open(cached_download(hf_hub_url(repo_id, filename, repo_type="dataset")), "r"))
id2label = {int(k): v for k, v in id2label.items()}
label2id = {v: k for k, v in id2label.items()}
num_labels = len(id2label)

In [7]:
from transformers import AutoImageProcessor

checkpoint = "nvidia/mit-b0"
image_processor = AutoImageProcessor.from_pretrained(checkpoint)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
from torchvision.transforms import ColorJitter

jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1)

In [9]:
def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch["image"]]
    labels = [x for x in example_batch["annotation"]]
    inputs = image_processor(images, labels)
    return inputs

In [10]:
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [11]:
import evaluate

metric = evaluate.load("mean_iou")

In [12]:
import torch
import torch.nn as nn
import numpy as np 

def compute_metrics(eval_pred):
    with torch.no_grad():
        logits, labels = eval_pred
        logits_tensor = torch.from_numpy(logits)
        logits_tensor = nn.functional.interpolate(
            logits_tensor,
            size=labels.shape[-2:],
            mode="bilinear",
            align_corners=False,
        ).argmax(dim=1)

        pred_labels = logits_tensor.detach().cpu().numpy()
        metrics = metric.compute(
            predictions=pred_labels,
            references=labels,
            num_labels=num_labels,
            ignore_index=255,
            reduce_labels=False,
        )
        for key, value in metrics.items():
            if type(value) is np.ndarray:
                metrics[key] = value.tolist()
        return metrics

In [13]:
from transformers import AutoModelForSemanticSegmentation, TrainingArguments, Trainer

model = AutoModelForSemanticSegmentation.from_pretrained(checkpoint, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at nvidia/mit-b0 were not used when initializing SegformerForSemanticSegmentation: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing SegformerForSemanticSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b0 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.3.proj.b

In [15]:
training_args = TrainingArguments(
    output_dir="segformer-b0-scene-parse-150",
    learning_rate=6e-5,
    num_train_epochs=50,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    eval_steps=100,
    logging_steps=1,
    eval_accumulation_steps=5,
    remove_unused_columns=False,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Cloning https://huggingface.co/Mustafa21/segformer-b0-scene-parse-150 into local empty directory.
***** Running training *****
  Num examples = 49
  Num Epochs = 50
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 1250
  Number of trainable parameters = 3752694


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Per Category Iou,Per Category Accuracy
100,3.579000,4.138644,0.033342,0.072470,0.308783,"[0.06580055095662675, 0.34785089422717685, 0.2744065522396124, 0.760051925327976, 0.26374943358340286, 0.04979291813326523, 0.16430853978046717, 0.0, 0.0, 0.013600273200717152, 0.0, 0.030700190985055137, 0.0, 0.08349348457860692, 0.0, 0.0, 0.04156576343954592, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, nan, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, nan, nan, nan, 0.0, 0.005223982518437586, nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, 0.0, nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0]","[0.11262633498765716, 0.6281159732772887, 0.46351564104319254, 0.9583615289740532, 0.49121370266291986, 0.8362829382914048, 0.26300703285753774, 0.0, 0.0, 0.015955209230584324, 0.0, 0.2710168930109308, 0.0, 0.09049112869426064, nan, 0.0, 0.061528197125442434, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, nan, nan, 0.0, nan, 0.0, nan, 0.0, nan, nan, nan, nan, 0.0, 0.011145104895104896, nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, 0.0, nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0]"
200,3.067100,3.448472,0.041143,0.079943,0.347148,"[0.04880581365501219, 0.28611369694329464, 0.4226913441260881, 0.7676016467854346, 0.293271350086148, 0.08166525444848727, 0.04463570322098179, 0.0, 0.0, 0.06244885866651618, 0.21411810715682442, 0.09441377915209664, 0.0, 0.1498418266670843, 0.0, 0.0, 0.0029784675920865667, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, nan, nan, 0.0, nan, 0.0, nan, 0.0, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, 0.0, nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0]","[0.07904583173812046, 0.8000120032141941, 0.8110031107156167, 0.9936390634535979, 0.4023961818100718, 0.6110997146746732, 0.04562314666630268, 0.0, 0.0, 0.08867009875603453, 0.29456599713055953, 0.32464392182842, 0.0, 0.18227770933614437, nan, 0.0, 0.0036977982834074026, 0.0, 0.0, 0.0, 0.0, 0.0, nan, 0.0, nan, 0.0, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, nan, nan, 0.0, nan, 0.0, nan, 0.0, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, nan, nan, 0.0, nan, nan, 0.0, nan, 0.0, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.0, nan, nan, 0.0, 0.0, nan, nan, nan, 0.0, 0.0, nan, 0.0, nan, 0.0, 0.0]"
300,3.391600,3.328

***** Running Evaluation *****
  Num examples = 21
  Batch size = 2
Saving model checkpoint to segformer-b0-scene-parse-150/checkpoint-100
Configuration saved in segformer-b0-scene-parse-150/checkpoint-100/config.json
Model weights saved in segformer-b0-scene-parse-150/checkpoint-100/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21
  Batch size = 2
Saving model checkpoint to segformer-b0-scene-parse-150/checkpoint-200
Configuration saved in segformer-b0-scene-parse-150/checkpoint-200/config.json
Model weights saved in segformer-b0-scene-parse-150/checkpoint-200/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21
  Batch size = 2
Saving model checkpoint to segformer-b0-scene-parse-150/checkpoint-300
Configuration saved in segformer-b0-scene-parse-150/checkpoint-300/config.json
Model weights saved in segformer-b0-scene-parse-150/checkpoint-300/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 21
  Batch size = 2
Saving model checkpoint

TrainOutput(global_step=1250, training_loss=2.3006627324581146, metrics={'train_runtime': 1507.5351, 'train_samples_per_second': 1.625, 'train_steps_per_second': 0.829, 'total_flos': 4.33832081227776e+16, 'train_loss': 2.3006627324581146, 'epoch': 50.0})

In [16]:
trainer.push_to_hub()

Saving model checkpoint to segformer-b0-scene-parse-150
Configuration saved in segformer-b0-scene-parse-150/config.json
Model weights saved in segformer-b0-scene-parse-150/pytorch_model.bin


Upload file pytorch_model.bin:   0%|          | 32.0k/14.4M [00:00<?, ?B/s]

remote: Scanning LFS files for validity...        
remote: LFS file scan complete.        
To https://huggingface.co/Mustafa21/segformer-b0-scene-parse-150
   781ab2f..d522cd3  main -> main

Dropping the following result as it does not have all the necessary fields:
{'dataset': {'name': 'scene_parse_150', 'type': 'scene_parse_150', 'config': 'scene_parsing', 'split': 'train[:70]', 'args': 'scene_parsing'}}
To https://huggingface.co/Mustafa21/segformer-b0-scene-parse-150
   d522cd3..8f946df  main -> main



'https://huggingface.co/Mustafa21/segformer-b0-scene-parse-150/commit/d522cd38ce47f956f4e8141ba9f7498f7b5d1106'